<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/test/model1_cross_validation_%2BAnalisiPicchiAllaFine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
@File name: model1_cross_validation.ipynb
@Created on 2018-12-20
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''

!pip install liac-arff

import arff
import math

import numpy as np

from keras import optimizers
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, LeakyReLU, BatchNormalization, Activation
from keras.callbacks import EarlyStopping

from sklearn.utils import compute_class_weight
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

import sys
sys.path.append('local_modules')

#import postprocessing_sliding
#import postprocessing_Silvia
#import plotting
import postprocessing_CO2
#import postprocessing_sw_HL

Using TensorFlow backend.


In [0]:
#fix random seed for reproducibility
seed = 5
np.random.seed(seed)

In [0]:
'''
@Description: generate a multilayer perceptron with LeakyRelu as activation
function.
@param: 
  - shape : int, the shape of the input
  - n_features: int, the number of features given
'''

#MODELLO 1
#REGOLA: input/2, input, 2*input, 1
#layers TUTTE LE FEATURE: 57, 113, 226, 1 
#layers TIME CO2 TEMP: 21, 41, 82, 1 
#layers TIME CO2 TEMP PM25/TVOC: 30, 59, 118, 1 
#layers TIME CO2 TEMP PM25 TVOC: 39, 77, 154, 1 

def generate_model_leaky(shape, n_features):
  
  units_1 = int(n_features/2)
  units_2 = n_features
  units_3 = n_features*2
  
  model = Sequential()
  model.add(BatchNormalization())
  
  model.add(Dense(units_1, input_dim=shape, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(units_2, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(units_3, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(1, activation='sigmoid'))
  #print(model.summary())

  return model

In [0]:
'''
@Description: generate a multilayer perceptron with Relu as activation
function.
@param: 
  - shape : int, the shape of the input
  - n_features: int, the number of features given
'''

#MODELLO 2
#REGOLA: a= input, b= a*2/3+c, c= b*2/3+1
#layers TUTTE LE FEATURE: 113, 229, 153, 1
#layers TIME CO2 TEMP: 41, 85, 57, 1 
#layers TIME CO2 TEMP PM25/TVOC: 59, 121, 81, 1 
#layers TIME CO2 TEMP PM25 TVOC: 77, 157, 105, 1 

def generate_model(shape, n_features):
  
  a = np.array([[1,0,0],[-(2/3),1,-1],[0,-(2/3),1]])
  b = np.array([n_features,0,1])
  x = np.linalg.solve(a, b)
  
  units_1 = int(x[0])
  units_2 = int(x[1])
  units_3 = int(x[2])
  
  model = Sequential()
 
  model.add(Dense(units_1, input_dim=shape, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0,5))
  
  model.add(Dense(units_2, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0,5))
  
  model.add(Dense(units_3, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0,5))
  
  model.add(Dense(1, activation='sigmoid'))
  #print(model.summary())

  return model

In [0]:
#@title SCEGLI I PARAMETRI

'''
@Description: MAIN
'''

#LOAD DATA
print("Loading data...")

dataset = '/root/data/uHooComplete_featureDataset.arff' #@param {type:"string"}

with open (dataset, encoding='utf-8') as f:
  dataDictionary = arff.load(f)

data = np.array(dataDictionary['data'])
print("DATASET LOADED")

#CONVERTING VALUES
print("\nConverting values...")
for i in data:
  if(i[-1] == 'Other'): i[-1] = 0
  elif(i[-1] == 'Pasto') : i[-1] = 1

dataset = data.astype('float32')
print("CONVERSION DONE")

#SPLIT INTO INPUT (X) AND OUTPUT (Y) VARIABLES
s = dataset.shape[-1]
X = dataset[:,0:s-1]
Y = dataset[:,s-1]

n_features = s-1

#OPTIMIZERS
adm = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

#LOSS
loss = 'binary_crossentropy'

#MODEL
modello = 1 #@param {type:"integer"}

#DEFINE K-FOLD CROSS-VALIDATION
fold = 10 #@param {type:"integer"}

kfold = KFold(n_splits=fold, shuffle=False, random_state=None)

cvscores = []
predictions = []

test_time =[]

real = []
true_p =[]
true_n = []

dimSplit = math.floor(len(dataset[:,0])/fold)
startIndex = 0
finishIndex = dimSplit-1

i = 1

for train, test in kfold.split(X, Y):
  
  print("\nFOLD: %d" %i)
  
  #COMPUTE CLASS WEIGHT
  labels = np.unique(Y[train])
  classWeight = compute_class_weight('balanced', labels, Y[train])
  classWeight = dict(zip(labels,classWeight))

  #GENERATE MODEL
  print("\nGenerate model...")

  if modello==1 :
    model = generate_model_leaky(X[train].shape[-1], n_features)
  elif modello ==2:
    model = generate_model(X[train].shape[-1], n_features)

  #COMPILE MODEL
  print("\nCompile model...")
  model.compile(loss = loss, optimizer = adm , metrics=['accuracy'])

  #FIT MODEL
  print("\nFit model...")
  epoche = 7 #@param{type:"integer"}
  history = model.fit(X[train], Y[train], epochs=epoche, batch_size = 128, shuffle = True, verbose=1, class_weight = classWeight)

  #EVALUATE MODEL
  print("\nEvaluate model...")
  scores_test = model.evaluate(X[test], Y[test], batch_size= 128, verbose = 0)
  print("Test loss: %.2f%%" % (scores_test[0] * 100))
  print("Test accuracy: %.2f%%" % (scores_test[1] * 100))
  
  cvscores.append(scores_test[1] * 100)
  
  #CALCULATE PREDICTIONS
  print("\nCalculate predictions...")
  pred = model.predict_classes(X[test], batch_size=128, verbose=0)
  flat_pred = [item for sublist in pred for item in sublist]
  predictions.append(flat_pred)
  
  real.append(Y[test])
  
  #STORE DATETIME
  time = []  
  for j in X[test]:
    time.append(int(j[-5]))  
  test_time.append(time)
  
  #PLOTTING
  #plotting.plot_model_results(history)
  #plotting.plot_co2_temp_cross(flat_pred, dimSplit, startIndex, finishIndex, i)
  
  #CONFUSION MATRIX
  print("\nCompute confusion matrix...")
  y_true = Y[test]
  tn, fp, fn, tp = confusion_matrix(y_true, pred).ravel()
  (tn, fp, fn, tp)
  other = 100*tn/(tn+fp)
  pasto = 100*tp/(fn+tp)
  true_p.append(pasto)
  true_n.append(other)
  print("Other: %.2f %%" % other)
  print("Pasto: %.2f %%" % pasto)
  
  i+=1

  
print("MEAN ACCURACY: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("MEAN TRUE POSITIVE RATE: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(true_p), np.std(true_p)))
print("MEAN TRUE NEGATIVE RATE: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(true_n), np.std(true_n)))


Loading data...
DATASET LOADED

Converting values...
CONVERSION DONE
Instructions for updating:
Colocations handled automatically by placer.

FOLD: 1

Generate model...

Compile model...

Fit model...
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/7
315441/315441 [==============================] - 18s 56us/step - loss: 0.5214 - acc: 0.7733
Epoch 2/7
315441/315441 [==============================] - 16s 51us/step - loss: 0.4154 - acc: 0.8367
Epoch 3/7
315441/315441 [==============================] - 16s 51us/step - loss: 0.3910 - acc: 0.8335
Epoch 4/7
315441/315441 [==============================] - 16s 51us/step - loss: 0.3823 - acc: 0.8301
Epoch 5/7
315441/315441 [==============================] - 16s 51us/step - loss: 0.3756 - acc: 0.8291
Epoch 6/7
315441/315441 [==============================] - 16s 51us/step - loss: 0.3691 - acc: 0.8292
Epoch 7/7
315441/315441 

In [0]:
import postprocessing_CO2

j = 0
results = []
for i in predictions:
  new_pred = postprocessing_CO2.get_postprocessing_CO2(i, j)
  results.append(new_pred)
  j = j + len(new_pred) -1
  
#CONFUSION MATRIX
print("\n Confusion matrix totale...")
y_true = [item for sublist in real for item in sublist]
y_true = np.array(y_true,dtype=np.int)
n_y_pred = [item for sublist in predictions for item in sublist]
tn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
other = 100*tn/(tn+fp)
pasto = 100*tp/(fn+tp)
print("Other corretti: %.2f %%" % other)
print("Pasto corretti: %.2f %%" % pasto)
  
#CONFUSION MATRIX POST PROCESSING
print("\n Confusion matrix post postprocessing...")
y_true = [item for sublist in real for item in sublist]
y_true = np.array(y_true,dtype=np.int)
n_y_pred = [item for sublist in results for item in sublist]
tn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
other = 100*tn/(tn+fp)
pasto = 100*tp/(fn+tp)
print("Other corretti: %.2f %%" % other)
print("Pasto corretti: %.2f %%" % pasto)

35050
35049
35049
35049
35049
35049
35049
35049
35049
35049

 Confusion matrix totale...
TN 289815
FP 44352
FN 3534
TP 12790
Other corretti: 86.73 %
Pasto corretti: 78.35 %

 Confusion matrix post postprocessing...
TN 305303
FP 28864
FN 6706
TP 9618
Other corretti: 91.36 %
Pasto corretti: 58.92 %


In [0]:
#@title SCEGLI maxWidth
'''
@Description: POST PROCESSING
'''
"""
maxWidth = 6 #@param {type:"integer"}

flat_pred = predictions
flat_pred = [item for sublist in flat_pred for item in sublist]

#CONFUSION MATRIX
print("\n Confusion matrix pre postprocessing...")
y_true = [item for sublist in real for item in sublist]
y_true = np.array(y_true,dtype=np.int)
n_y_pred = flat_pred 
tn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
other = 100*tn/(tn+fp)
pasto = 100*tp/(fn+tp)
print("Other corretti: %.2f %%" % other)
print("Pasto corretti: %.2f %%" % pasto)

#new_pred = postprocessing_sliding.sliding_windows(flat_pred)
new_pred = postprocessing_Silvia.setWidth(maxWidth,flat_pred)

#CONFUSION MATRIX
print("\n Confusion matrix post postprocessing...")
y_true = [item for sublist in real for item in sublist]
y_true = np.array(y_true,dtype=np.int)
n_y_pred = new_pred 
tn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
other = 100*tn/(tn+fp)
pasto = 100*tp/(fn+tp)
print("Other corretti: %.2f %%" % other)
print("Pasto corretti: %.2f %%" % pasto)
"""

'\nmaxWidth = 6 #@param {type:"integer"}\n\nflat_pred = predictions\nflat_pred = [item for sublist in flat_pred for item in sublist]\n\n#CONFUSION MATRIX\nprint("\n Confusion matrix pre postprocessing...")\ny_true = [item for sublist in real for item in sublist]\ny_true = np.array(y_true,dtype=np.int)\nn_y_pred = flat_pred \ntn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()\nprint("TN", tn)\nprint("FP", fp)\nprint("FN", fn)\nprint("TP", tp)\nother = 100*tn/(tn+fp)\npasto = 100*tp/(fn+tp)\nprint("Other corretti: %.2f %%" % other)\nprint("Pasto corretti: %.2f %%" % pasto)\n\n#new_pred = postprocessing_sliding.sliding_windows(flat_pred)\nnew_pred = postprocessing_Silvia.setWidth(maxWidth,flat_pred)\n\n#CONFUSION MATRIX\nprint("\n Confusion matrix post postprocessing...")\ny_true = [item for sublist in real for item in sublist]\ny_true = np.array(y_true,dtype=np.int)\nn_y_pred = new_pred \ntn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()\nprint("TN", tn)\nprint("FP", f

In [0]:
flat_pred = predictions
test_label = [item for sublist in flat_pred for item in sublist]

print(test_label)
time = test_time

#@title IMPOSTA LIMITI DELLA FINESTRA DI: COLAZIONE, PRANZO E CENA 
lim_inf_colazione = 330 #@param {type:"integer"}
lim_sup_colazione = 630 #@param {type:"integer"}
lim_inf_pranzo = 720 #@param {type:"integer"}
lim_sup_pranzo = 900 #@param {type:"integer"}
lim_inf_cena = 1111 #@param {type:"integer"}
lim_sup_cena = 1439 #@param {type:"integer"}

postprocessing_sw_HL.confusion_matrix_activity_pre_sw(test_label, pred,time,lim_inf_colazione,lim_sup_colazione,lim_inf_pranzo,lim_sup_pranzo,lim_inf_cena,lim_sup_cena)

#@title IMPOSTA LA GRANDEZZA DELLA FINESTRA DI: COLAZIONE, PRANZO E CENA 
finestra_colazione = 5 #@param {type:"integer"}
finestra_pranzo = 31 #@param {type:"integer"}
finestra_cena = 31 #@param {type:"integer"}
min_pasti_colazione = 3 #@param {type:"integer"}
min_pasti_pranzo = 15 #@param {type:"integer"}
min_pasti_cena = 15 #@param {type:"integer"}

sw_real_activity = postprocessing_sw_HL.sw_for_real_activity(test_label,time,lim_inf_colazione,lim_sup_colazione,lim_inf_pranzo,lim_sup_pranzo,lim_inf_cena,lim_sup_cena,finestra_colazione,finestra_pranzo,finestra_cena)
sw_col_real = sw_real_activity[0]
sw_pranzo_real = sw_real_activity[1]
sw_cena_real = sw_real_activity[2]

sw_pred_activity = postprocessing_sw_HL.sw_for_pred_activity(pred,time,lim_inf_colazione,lim_sup_colazione,lim_inf_pranzo,lim_sup_pranzo,lim_inf_cena,lim_sup_cena,finestra_colazione,finestra_pranzo,finestra_cena, min_pasti_colazione, min_pasti_pranzo, min_pasti_cena)
result_col = sw_pred_activity[0]
result_pranzo = sw_pred_activity[1]
result_cena = sw_pred_activity[2]

postprocessing_sw_HL.confusion_matrix_activity_post_sw(sw_col_real, sw_pranzo_real, sw_cena_real,result_col, result_pranzo, result_cena)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

NameError: ignored